In [3]:
# #Install Necessary Libraries
# %pip install pandas
# %pip install --upgrade pandas
# %pip install --upgrade openpyxl

import pandas as pd
import tkinter as tk
from tkinter import filedialog


In [ ]:
# Create a function to open a file dialog and import the XLSX file

def open_excel_file():
    # Initialize Tkinter root
    root = tk.Tk()
    root.withdraw()  # Hide the root window
    
    # Open file dialog and ask the user to select a XLSX file
    file_path = filedialog.askopenfilename(
        title="Select an excel file",
        filetypes=[("Excel files", "*.xlsx")]
    )
    
    # Check if a file was selected
    if file_path:
        # Load the selected CSV file into a pandas DataFrame
        df = pd.read_excel(file_path)
        print(f"File '{file_path}' successfully loaded into a DataFrame.")
        return df
    else:
        print("No file was selected.")
        return None

# Call the function and store the DataFrame in a variable
df = open_excel_file()

# Now you can use the 'data' variable in your subsequent code
if df is not None:
    # Example: Print the first few rows of the DataFrame
    print(df.head(2))
else:
    print("Data could not be loaded.")

# Delete the first two columns and the first row
df = df.iloc[1:, 1:]

#Set new header as first row
new_header = df.iloc[0]
df = df[1:]
df.columns = new_header

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Extract column headers
column_headers = df.columns.tolist()

# Create checkboxes for each column header
checkboxes = [widgets.Checkbox(value=False, description=col) for col in column_headers]

# Create a button to apply the selection
button = widgets.Button(description="Apply Selection")
output = widgets.Output()

# Function to retrieve selected columns based on checkboxes
def get_selected_columns():
    selected_columns = [checkbox.description for checkbox in checkboxes if checkbox.value]
    return selected_columns

# Define a global variable to store the filtered DataFrame
global saved_filtered_df
saved_filtered_df = None

def on_button_clicked(b):
    global saved_filtered_df
    with output:
        # Clear previous output
        output.clear_output()

        # Get the selected columns
        selected_columns = get_selected_columns()

        if selected_columns:
            # Filter the DataFrame by selected columns
            saved_filtered_df = df[selected_columns]
            
            # Display the filtered DataFrame
            display(saved_filtered_df)

            # Create a new column that averages the selected trials
            trial_columns = [col for col in saved_filtered_df.columns if 'Trial' in col]
            if trial_columns:
                saved_filtered_df['Average'] = saved_filtered_df[trial_columns].mean(axis=1)
                display(saved_filtered_df)
        else:
            print("No columns selected.")

# Attach the function to the button
button.on_click(on_button_clicked)

# Display checkboxes, button, and output
display(*checkboxes, button, output)

In [ ]:
cmj = saved_filtered_df.transpose()

print(cmj.head(1))

In [ ]:
# Step 2: Extract the header row and the trial column
avg = cmj.iloc[3]
headers = cmj.iloc[0]  # Extract headers from the first row, excluding the first column
trials = (['Trial 1', 'Trial 2'])  # Extract the trial numbers from the index, excluding the header and last row

# Step 3: Create new headers by combining each header with each trial
new_headers = [f"{header}_{trial}" for trial in trials for header in headers]

# Step 4: Replicate the headers and trials into the final 2-row DataFrame
data = cmj.iloc[ 1:3].values.flatten()

# Concatenate the header and trial rows to create the final DataFrame
final_df = pd.DataFrame([new_headers, data], index=['Header', 'Data'])

# Transpose the DataFrame to have the headers as columns
final_df = final_df.T

# Create your average datafram
headers = pd.Series(headers)
average = pd.concat([headers.reset_index(drop=True), avg.reset_index(drop=True)], axis=1)
average = average.transpose()
final_df = final_df.transpose()
print(final_df.head(2))

In [ ]:
def save_final_file(dataframe):
    file_path = filedialog.asksaveasfilename(
        title="Save CSV file",
        defaultextension=".csv",
        filetypes=[("CSV files", "*.csv")]
    )

    if file_path:
        final_df.to_csv(file_path, index=False, header=False)
        print(f"File successfully saved as '{file_path}'.")
    else: 
        print("File not saved.")

def save_average_file(dataframe):
    file_path = filedialog.asksaveasfilename(
        title="Save CSV file",
        defaultextension=".csv",
        filetypes=[("CSV files", "*.csv")]
    )

    if file_path:
        average.to_csv(file_path, index=False, header=False)
        print(f"File successfully saved as '{file_path}'.")
    else: 
        print("File not saved.")

save_final_file(final_df)
save_average_file(average)